In [11]:
import pandas as pd
import numpy as np
import pickle

DATA_PATH = '../data/'

X = pd.read_csv(DATA_PATH + "X_model_selection.csv")
y = pd.read_csv(DATA_PATH + "y_model_selection.csv")

with open(DATA_PATH + "determined_feature_sets.pkl", "rb") as f:
    determined_feature_sets = pickle.load(f)

In [12]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    average_precision_score, roc_auc_score,
    accuracy_score, recall_score, precision_score, fbeta_score,
    confusion_matrix
)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB

SEED = 42

y_vec = y.squeeze() if hasattr(y, "squeeze") else y
if isinstance(y_vec, pd.DataFrame):
    y_vec = y_vec.iloc[:, 0]

# Train/Val/Test split (70/15/15)
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y_vec, test_size=0.15, stratify=y_vec, random_state=SEED
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.1764706,  # 0.176... of 85% ≈ 15%
    stratify=y_trainval, random_state=SEED
)

print("Shapes:", X_train.shape, X_val.shape, X_test.shape)
print("Pos rates:", y_train.mean(), y_val.mean(), y_test.mean())

Shapes: (1028, 19) (221, 19) (221, 19)
Pos rates: 0.1605058365758755 0.16289592760180996 0.16289592760180996


In [13]:
def get_scores(estimator, X_part):
    """Return continuous scores for PR-AUC/ROC-AUC and thresholding."""
    if hasattr(estimator, "predict_proba"):
        return estimator.predict_proba(X_part)[:, 1]
    if hasattr(estimator, "decision_function"):
        return estimator.decision_function(X_part)
    raise AttributeError("Estimator has neither predict_proba nor decision_function")

def pick_threshold_max_f2(y_true, scores, grid_size=200):
    thresholds = np.linspace(0.01, 0.99, grid_size)
    best_t, best_f2 = 0.5, -1.0
    for t in thresholds:
        y_pred = (scores >= t).astype(int)
        f2 = fbeta_score(y_true, y_pred, beta=2, zero_division=0)
        if f2 > best_f2:
            best_f2, best_t = f2, t
    return float(best_t), float(best_f2)

def eval_metrics(y_true, scores, threshold):
    y_pred = (scores >= threshold).astype(int)
    return {
        "pr_auc": float(average_precision_score(y_true, scores)),
        "roc_auc": float(roc_auc_score(y_true, scores)),
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "recall": float(recall_score(y_true, y_pred, zero_division=0)),
        "precision": float(precision_score(y_true, y_pred, zero_division=0)),
        "f2": float(fbeta_score(y_true, y_pred, beta=2, zero_division=0)),
        "tn_fp_fn_tp": tuple(confusion_matrix(y_true, y_pred).ravel())
    }


---

## Final evaluation metrics and threshold selection

Model performance was evaluated using a combination of **threshold-independent** and **threshold-dependent** metrics to reflect both ranking quality and real-world decision-making.

### Threshold-independent metrics

* **PR-AUC (Precision–Recall AUC)** was used as the primary metric for model comparison. PR-AUC is well-suited to imbalanced classification problems because it focuses on performance on the minority class and does not depend on a fixed decision threshold.
* **ROC-AUC** was reported as a secondary metric to assess overall ranking performance and class separability.

These metrics evaluate how well a model ranks attrition cases without committing to a specific classification rule.

---

### Threshold-dependent metrics

For operational use, a fixed decision threshold is required. To evaluate this setting:

* **Recall** measures the proportion of actual attrition cases correctly identified and directly reflects the cost of missed attrition events.
* **Precision** quantifies the proportion of predicted attrition cases that are correct, reflecting the cost of false positives.
* **Accuracy** is reported for completeness but is not relied upon due to class imbalance.

---

### F2-score and threshold selection

The **F2-score** was used as the primary threshold-dependent metric. Unlike F1, F2 places greater emphasis on recall, aligning with the objective of minimizing missed attrition cases while still penalizing excessive false positives.

Decision thresholds were selected by **maximizing F2-score on the validation set only**. This ensures that:

* threshold selection is aligned with business priorities,
* and the test set remains untouched until final evaluation.

---


In [14]:
models_to_tune = {
    "logreg_l2": ("linear",
        LogisticRegression(
            penalty="l2", solver="lbfgs", class_weight="balanced",
            max_iter=5000, random_state=SEED
        )
    ),
    "logreg_l1": ("linear",
        LogisticRegression(
            penalty="l1", solver="liblinear", class_weight="balanced",
            max_iter=5000, random_state=SEED
        )
    ),
    "linearsvc": ("linear",
        LinearSVC(class_weight="balanced", random_state=SEED)
    ),
    "gaussian_nb": ("naive_bayes",
        GaussianNB()
    ),
}

# Param grids
param_grids = {
    "logreg_l2": {"C": [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30]},
    "logreg_l1": {"C": [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30]},
    "linearsvc": {"C": [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30]},
    "gaussian_nb": {"var_smoothing": [1e-12, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6]},
}

# CV for tuning
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

In [15]:
best_models = {}
metrics_rows = []
preds_val_rows = []
preds_test_rows = []

for model_name, (family, base_est) in models_to_tune.items():
    feats = determined_feature_sets[family]
    
    Xtr = X_train[feats]
    Xva = X_val[feats]
    Xte = X_test[feats]

    grid = GridSearchCV(
        estimator=base_est,
        param_grid=param_grids[model_name],
        scoring="average_precision",
        cv=inner_cv,
        n_jobs=16,
        verbose=0,
        refit=True
    )
    grid.fit(Xtr, y_train)
    best_est = grid.best_estimator_

    best_est.fit(Xtr, y_train)
    best_models[model_name] = best_est

    # Scores
    tr_scores = get_scores(best_est, Xtr)
    va_scores = get_scores(best_est, Xva)
    te_scores = get_scores(best_est, Xte)

    best_t, best_f2_val = pick_threshold_max_f2(y_val, va_scores)

    for split_name, y_true, scores in [
        ("train", y_train, tr_scores),
        ("val",   y_val,   va_scores),
        ("test",  y_test,  te_scores),
    ]:
        row = {
            "model": model_name,
            "family": family,
            "split": split_name,
            "threshold": best_t,
            "best_params": grid.best_params_,
        }
        row.update(eval_metrics(y_true, scores, best_t))
        metrics_rows.append(row)

    preds_val_rows.append(pd.DataFrame({
        "model": model_name,
        "family": family,
        "y_true": y_val.values if hasattr(y_val, "values") else y_val,
        "score": va_scores,
        "threshold": best_t,
        "y_pred": (va_scores >= best_t).astype(int),
    }))

    preds_test_rows.append(pd.DataFrame({
        "model": model_name,
        "family": family,
        "y_true": y_test.values if hasattr(y_test, "values") else y_test,
        "score": te_scores,
        "threshold": best_t,
        "y_pred": (te_scores >= best_t).astype(int),
    }))

metrics_df = pd.DataFrame(metrics_rows)

preds_val_df = pd.concat(preds_val_rows, ignore_index=True)
preds_test_df = pd.concat(preds_test_rows, ignore_index=True)

metrics_df.sort_values(["split", "pr_auc"], ascending=[True, False])


,model,family,split,threshold,best_params,pr_auc,roc_auc,accuracy,recall,precision,f2,tn_fp_fn_tp
5,logreg_l1,linear,test,0.467990,{'C': 0.3},0.625344,0.823423,0.742081,0.777778,0.363636,0.633484,"(136, 49, 8, 28)"
8,linearsvc,linear,test,0.029698,{'C': 0.01},0.617437,0.827177,0.773756,0.750000,0.397059,0.636792,"(144, 41, 9, 27)"
2,logreg_l2,linear,test,0.458141,{'C': 0.1},0.613357,0.824024,0.714932,0.777778,0.337349,0.616740,"(130, 55, 8, 28)"
11,gaussian_nb,naive_bayes,test,0.103568,{'var_smoothing': 1e-07},0.611212,0.832583,0.656109,0.861111,0.303922,0.630081,"(114, 71, 5, 31)"
9,gaussian_nb,naive_bayes,train,0.103568,{'var_smoothing': 1e-07},0.631025,0.848330,0.666342,0.866667,0.308190,0.636121,"(542, 321, 22, 143)"
0,logreg_l2,linear,train,0.458141,{'C': 0.1},0.625686,0.853120,0.728599,0.824242,0.352332,0.650096,"(613, 250, 29, 136)"
6,linearsvc,linear,train,0.029698,{'C': 0.01},0.622382,0.852607,0.777237,0.793939,0.401840,0.664300,"(668, 195, 34, 131)"
3,logreg_l1,linear,train,0.467990,{'C': 0.3},0.615694,0.853246,0.742218,0.806061,0.363388,0.648148,"(630, 233, 32, 133)"
4,logreg_l1,linear,val,0.467990,{'C': 0.3},0.571817,0.808709,0.714932,0.777778,0.337349,0.616740,"(130, 55, 8, 28)"
10,gaussian_nb,naive_bayes,val,0.103568,{'var_smoothing': 1e-07},0.568735,0.809159,0.669683,0.833333,0.309278,0.622407,"(118, 67, 6, 30)"


In [16]:
test_leaderboard = (
    metrics_df[metrics_df["split"] == "test"]
    .sort_values(["f2", "recall", "pr_auc"], ascending=False)
    .reset_index(drop=True)
)

test_leaderboard

,model,family,split,threshold,best_params,pr_auc,roc_auc,accuracy,recall,precision,f2,tn_fp_fn_tp
0,linearsvc,linear,test,0.029698,{'C': 0.01},0.617437,0.827177,0.773756,0.750000,0.397059,0.636792,"(144, 41, 9, 27)"
1,logreg_l1,linear,test,0.467990,{'C': 0.3},0.625344,0.823423,0.742081,0.777778,0.363636,0.633484,"(136, 49, 8, 28)"
2,gaussian_nb,naive_bayes,test,0.103568,{'var_smoothing': 1e-07},0.611212,0.832583,0.656109,0.861111,0.303922,0.630081,"(114, 71, 5, 31)"
3,logreg_l2,linear,test,0.458141,{'C': 0.1},0.613357,0.824024,0.714932,0.777778,0.337349,0.616740,"(130, 55, 8, 28)"


In [17]:
tp_counts = (
    preds_test_df
    .query("y_true == 1 and y_pred == 1")
    .groupby("model")
    .size()
    .rename("true_positives")
    .reset_index()
)

total_positives = preds_test_df.query("y_true == 1").groupby("model").size()

summary = (
    tp_counts
    .set_index("model")
    .join(total_positives.rename("total_positives"))
    .assign(
        recall=lambda df: df["true_positives"] / df["total_positives"]
    )
    .reset_index()
)

summary


,model,true_positives,total_positives,recall
0,gaussian_nb,31,36,0.861111
1,linearsvc,27,36,0.750000
2,logreg_l1,28,36,0.777778
3,logreg_l2,28,36,0.777778


In [18]:
val_leaderboard = (
    metrics_df[metrics_df["split"] == "val"]
    .sort_values(["f2", "recall", "pr_auc"], ascending=False)
    .reset_index(drop=True)
)

val_leaderboard

,model,family,split,threshold,best_params,pr_auc,roc_auc,accuracy,recall,precision,f2,tn_fp_fn_tp
0,gaussian_nb,naive_bayes,val,0.103568,{'var_smoothing': 1e-07},0.568735,0.809159,0.669683,0.833333,0.309278,0.622407,"(118, 67, 6, 30)"
1,logreg_l2,linear,val,0.458141,{'C': 0.1},0.544280,0.806607,0.723982,0.777778,0.345679,0.622222,"(132, 53, 8, 28)"
2,logreg_l1,linear,val,0.467990,{'C': 0.3},0.571817,0.808709,0.714932,0.777778,0.337349,0.616740,"(130, 55, 8, 28)"
3,linearsvc,linear,val,0.029698,{'C': 0.01},0.557982,0.809459,0.751131,0.722222,0.366197,0.604651,"(140, 45, 10, 26)"


In [19]:
tp_counts = (
    preds_val_df
    .query("y_true == 1 and y_pred == 1")
    .groupby("model")
    .size()
    .rename("true_positives")
    .reset_index()
)

total_positives = preds_val_df.query("y_true == 1").groupby("model").size()

summary = (
    tp_counts
    .set_index("model")
    .join(total_positives.rename("total_positives"))
    .assign(
        recall=lambda df: df["true_positives"] / df["total_positives"]
    )
    .reset_index()
)

summary


,model,true_positives,total_positives,recall
0,gaussian_nb,30,36,0.833333
1,linearsvc,26,36,0.722222
2,logreg_l1,28,36,0.777778
3,logreg_l2,28,36,0.777778



---

## Final model selection

Final model selection was based **exclusively on validation performance**, with the test set reserved for a one-time confirmation of generalization.

### Validation-based decision

Using the validation set:

* **Gaussian Naive Bayes** achieved the **highest F2-score** and **highest recall**
* It correctly identified **30 out of 36 attrition cases** (83.3% recall)
* Other linear models identified between **26–28** attrition cases

Because the primary objective is to **minimize missed attrition events**, validation recall and F2-score were prioritized over precision and accuracy.

| Model       | Validation Recall | Validation F2 | True Positives |
| ----------- | ----------------- | ------------- | -------------- |
| GaussianNB  | **0.833**         | **0.622**     | **30 / 36**    |
| LogReg (L2) | 0.778             | 0.622         | 28 / 36        |
| LogReg (L1) | 0.778             | 0.617         | 28 / 36        |
| LinearSVC   | 0.722             | 0.605         | 26 / 36        |

GaussianNB consistently missed the **fewest attrition cases**, aligning most closely with the stated business objective.

---

### Test set confirmation

The test set was used only to verify that the selected model generalizes:

* GaussianNB achieved **86.1% recall** on the test set
* It correctly identified **31 out of 36 attrition cases**
* No performance collapse or instability was observed

Differences in ranking between validation and test sets are expected due to sampling variability and threshold sensitivity and were **not** used for model selection.

---

### Final choice

Based on:

* superior validation recall and F2-score,
* stable performance across splits,
* and alignment with business priorities,

**Gaussian Naive Bayes was selected as the final model**.

This choice prioritizes minimizing missed attrition events while maintaining competitive overall performance.

---

## Limitations and potential improvements

### Limitations

1. **Limited sample size and class imbalance**
   Attrition events constitute a relatively small portion of the dataset. As a result, evaluation metrics—especially recall and F2-score—can vary noticeably across validation and test splits. While this is expected, it limits the statistical certainty of small performance differences between top models.

2. **Fixed decision threshold**
   A single decision threshold was selected to maximize F2-score on the validation set. In practice, optimal thresholds may vary over time or across departments depending on organizational capacity to act on predictions.

3. **Simplified cost assumptions**
   The analysis assumes that false negatives (missed attrition) are more costly than false positives. However, no explicit cost function was available, so trade-offs were evaluated using proxy metrics (recall and F2-score).

4. **Feature engineering scope**
   Feature selection relied on stability across folds and model families. While this improves robustness, more expressive features (e.g., temporal trends or interaction terms) were not explored.

5. **Probability calibration**
   Some models (e.g., LinearSVC) do not produce calibrated probabilities. While this does not affect ranking metrics, it may limit interpretability when predicted scores are used directly by stakeholders.

---

### Potential improvements

1. **Cost-sensitive learning**
   Incorporating an explicit cost matrix or utility-based objective could align model optimization more directly with real business consequences.

2. **Threshold optimization by operational constraints**
   Instead of maximizing F2-score alone, thresholds could be chosen based on constraints such as maximum allowable follow-ups or minimum recall requirements.

3. **Probability calibration and uncertainty estimation**
   Applying calibration techniques (e.g., Platt scaling or isotonic regression) would improve the interpretability of predicted attrition probabilities.

4. **Richer feature representations**
   Including temporal features (e.g., changes in satisfaction or workload over time) or interaction terms may improve predictive performance beyond static attributes.

5. **External validation**
   Evaluating the model on data from a different time period or organizational unit would provide stronger evidence of generalization.

---
